In [5]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [6]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Hermanus,ZA,-34.42,19.23,55.99,91,100,3.00,overcast clouds,0.0,0
1,1,Punta Arenas,CL,-53.15,-70.92,35.60,93,94,4.70,overcast clouds,0.0,0
2,2,Rikitea,PF,-23.12,-134.97,70.99,65,58,17.29,broken clouds,0.0,0
3,3,Port Alfred,ZA,-33.59,26.89,57.99,91,37,4.00,scattered clouds,0.0,0
4,4,Ushuaia,AR,-54.80,-68.30,35.60,80,56,5.82,broken clouds,0.0,0


In [7]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

# Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

In [8]:
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 60


In [9]:
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the maximum temperature you would like for your vacation? 90


In [10]:
rain_amount = input("Do you want it to be raining? (yes/no) ")

Do you want it to be raining? (yes/no) no


In [11]:
snow_amount = input("Do you want it to be snowing? (yes/no) ")

Do you want it to be snowing? (yes/no) no


In [13]:
if rain_amount == "no" and snow_amount == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] == 0) & \
                                        (city_data_df["Snow (inches)"] == 0)]
elif rain_amount == "no" and snow_amount == "yes":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] == 0) & \
                                        (city_data_df["Snow (inches)"] >= 0.0)]
elif rain_amount == "yes" and snow_amount == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] >= 0.0) & \
                                        (city_data_df["Snow (inches)"] == 0.0)]
else:
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] >= 0.0) & \
                                        (city_data_df["Snow (inches)"] >= 0.0)]

In [14]:
filtered_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
2,2,Rikitea,PF,-23.12,-134.97,70.99,65,58,17.29,broken clouds,0.0,0
5,5,Bathsheba,BB,13.22,-59.52,82.40,83,40,20.80,haze,0.0,0
6,6,Albany,US,42.60,-73.97,81.00,85,30,2.84,scattered clouds,0.0,0
8,8,Bethel,US,41.37,-73.41,73.40,73,1,6.93,clear sky,0.0,0
9,9,Komsomolskiy,UZ,40.43,71.72,71.60,35,0,4.70,clear sky,0.0,0
11,11,Khatanga,RU,71.97,102.50,66.33,50,83,9.26,broken clouds,0.0,0
12,12,Gazojak,TM,41.19,61.40,86.67,13,0,13.94,clear sky,0.0,0
13,13,Omboue,GA,-1.57,9.26,71.55,93,34,4.97,scattered clouds,0.0,0
14,14,Rafai,CF,4.95,23.92,67.33,98,12,2.39,few clouds,0.0,0
19,19,Puerto Ayora,EC,-0.74,-90.35,70.00,94,21,1.99,few clouds,0.0,0


In [15]:
len(filtered_cities_df)

334

In [16]:
filtered_cities_df.count()

City_ID                334
City                   334
Country                334
Lat                    334
Lng                    334
Max Temp               334
Humidity               334
Cloudiness             334
Wind Speed             334
Current Description    334
Rain (inches)          334
Snow (inches)          334
dtype: int64

In [29]:
filtered_cities_df = filtered_cities_df.dropna()

In [30]:
hotel_df = filtered_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Rikitea,PF,70.99,-23.12,-134.97,
5,Bathsheba,BB,82.40,13.22,-59.52,
6,Albany,US,81.00,42.60,-73.97,
8,Bethel,US,73.40,41.37,-73.41,
9,Komsomolskiy,UZ,71.60,40.43,71.72,


In [31]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate Through hotel_df DataFrame

In [32]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    # hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieva

In [34]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Rikitea,PF,70.99,-23.12,-134.97,Pension Maro'i
5,Bathsheba,BB,82.40,13.22,-59.52,Atlantis Hotel
6,Albany,US,81.00,42.60,-73.97,
8,Bethel,US,73.40,41.37,-73.41,Courtyard by Marriott Danbury
9,Komsomolskiy,UZ,71.60,40.43,71.72,Ikathouse
11,Khatanga,RU,66.33,71.97,102.50,"Gostinitsa ""Zapolyar'ye"""
12,Gazojak,TM,86.67,41.19,61.40,
13,Omboue,GA,71.55,-1.57,9.26,Hotel Olako
14,Rafai,CF,67.33,4.95,23.92,
19,Puerto Ayora,EC,70.00,-0.74,-90.35,Finch Bay Galapagos Hotel


In [35]:
hotel_df.count()

City          334
Country       334
Max Temp      334
Lat           334
Lng           334
Hotel Name    334
dtype: int64

In [36]:
len(hotel_df)

334

In [39]:
hotel_df = hotel_df.dropna()

In [41]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Rikitea,PF,70.99,-23.12,-134.97,Pension Maro'i
5,Bathsheba,BB,82.40,13.22,-59.52,Atlantis Hotel
6,Albany,US,81.00,42.60,-73.97,
8,Bethel,US,73.40,41.37,-73.41,Courtyard by Marriott Danbury
9,Komsomolskiy,UZ,71.60,40.43,71.72,Ikathouse
11,Khatanga,RU,66.33,71.97,102.50,"Gostinitsa ""Zapolyar'ye"""
12,Gazojak,TM,86.67,41.19,61.40,
13,Omboue,GA,71.55,-1.57,9.26,Hotel Olako
14,Rafai,CF,67.33,4.95,23.92,
19,Puerto Ayora,EC,70.00,-0.74,-90.35,Finch Bay Galapagos Hotel


In [42]:
output_data_file = "data/WeatherPy_vacation.csv"
hotel_df.to_csv(output_data_file, index_label = "City_ID")

In [43]:
vacation_df = pd.read_csv("data/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,2,Rikitea,PF,70.99,-23.12,-134.97,Pension Maro'i
1,5,Bathsheba,BB,82.40,13.22,-59.52,Atlantis Hotel
2,6,Albany,US,81.00,42.60,-73.97,NaN
3,8,Bethel,US,73.40,41.37,-73.41,Courtyard by Marriott Danbury
4,9,Komsomolskiy,UZ,71.60,40.43,71.72,Ikathouse


In [44]:
vacation_df = vacation_df.dropna()

In [45]:
vacation_df.head()

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,2,Rikitea,PF,70.99,-23.12,-134.97,Pension Maro'i
1,5,Bathsheba,BB,82.40,13.22,-59.52,Atlantis Hotel
3,8,Bethel,US,73.40,41.37,-73.41,Courtyard by Marriott Danbury
4,9,Komsomolskiy,UZ,71.60,40.43,71.72,Ikathouse
5,11,Khatanga,RU,66.33,71.97,102.50,"Gostinitsa ""Zapolyar'ye"""


# Using template add the hotel marks

In [46]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
             dissipating=False, max_intensity=300, point_radius=4)
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))